# Índice 2: positividad test RT-PCR

## Objetivo: disminuir positividad 

### Meta: Disminuir el índice de positividad por tramo de ubicación.

*Tramo 1* (>nivel nacional): baja pogresiva hasta alcanzar nivel nacional

*Tramo 2* (= nivel nacional y >10%): 10%

*Tramo 3* (menor nivel nacional y >10%): 10%

*Tramo 4* (<10%): 5%

*Tramo 5* (<5%) mantener <5%

**Periodicidad:** Semanal

**Escala territorial:** comunal, regional, nacional

**Fuente de información** base EPIVIGILA y registros gestión BAC, actualizadas con los reportes de laboratorio al día de la determinación del indicador

**Cálculo del indicador a nivel comunal**

$\frac{No.\:notificados\:c/test\:positivo\:por\:comuna\: de\:residencia\:semana\:x}{No.\:total\:notificados\:c/test\:por\:comuna\:de\: residencia\:semana\:x}$

**Cálculo del indicador a nivel regional**

$\frac{No.\:notificados\:c/test\:positivo\:por\:región\: de\:residencia\:semana\:x}{No.\:total\:notificados\:c/test\:por\:región\:de\: residencia\:semana\:x}$

**Observación** 

- Se considera notificación con test aquella notificación (o registro gestión BAC) que tiene resultado de test RT-PCR

## Importación de datos y librerias

In [1]:
# Importa librerias necesarias para el análisis

import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
# Importa base de datos obtenida mediante conexión FTPS 

data = pd.read_csv("data/20210823_Región_de_Aysén_del_General_Carlos_Ibáñez_del_Campo_notificaciones_0600.csv", 
                   sep=";", # CSV con separador de semicolon
                   low_memory=False
                  )

In [3]:
# Filtra datos obtenidos solo para pacientes con residencia en la Región de Aysén

data_aysen = data[data['region_residencia'] == 'Región de Aysén del General Carlos Ibáñez del Campo']

data_aysen.head(1)

,id_formulario_eno,numero_folio,id_enfermedad_eno,enfermedad_notificada,semana_epidemiologica,fecha_notificacion,etapa_clinica,establecimiento_salud,region,seremi,...,fecha_resultado_pcr_3,fecha_resultado_hemocultivo_3,fecha_resultado_otro_cultivo_3,factor_riesgo_caso_confirmado,factor_riesgo_caso_sospechoso,factor_riesgo_viaje_internacional,factor_riesgo_viaje_nacional,ningun_factor_riesgo,test_antigeno_sars_positivo,tac_torax_covid
0,100166,50166,50,Coronavirus (COVID-19),11,2020-03-12,DESCARTADA,Hospital Regional (Coihaique),Región de Aysén del General Carlos Ibáñez del ...,SEREMI De Aisén del General Carlos Ibañez del ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Variables de importancia


|Variable|Tipo|Descripción|Observación|
|---|---|---|---|
|`region_residencia`| Text | Residencia registrada | Variable utilizada pero no mostrada
|`numero_folio`|Numeric| id | el número de folio se mantendrá por trazabilidad
|`comuna_residencia`|Text| City | Comuna de residencia informada
|`tipo_identificacion`|Text| Tipo de documento de identidad | Carnet o Pasaporte
|`identificacion_paciente`|Numeric|Rol Unico Nominal o Numero de pasaporte| |
|`fecha_notificacion`|DateTime| Fecha de notificación| DateTime en formato `%Y-%m-%d` |
|`semana_epidemiologica`|Numeric|Semana epidemiologica|valores de 1 al 52|
|`etapa_clinica`|Text|`SOSPECHA` o `BUSQUEDA A`| Debe ser filtrada para evitar valores `DESCARTADA` |
|`resultado_pcr_1`|Text||
|`fecha_resultado_pcr_1`|DateTime||
|`fecha_toma_muestra_1`|DateTime||
|`resultado_pcr_2`|Text||
|`fecha_resultado_pcr_2`|DateTime||
|`fecha_toma_muestra_2`|DateTime||
|`resultado_pcr_3`|Text||
|`fecha_resultado_pcr_3`|DateTime||
|`fecha_toma_muestra_3`|DateTime||





In [4]:
# Selecciona variables de importancia


data_aysen_clean =  data_aysen[[
                                'numero_folio',
                                'comuna_residencia',
                                'fecha_notificacion',
                                'semana_epidemiologica',
                                'etapa_clinica',
                                'resultado_pcr_1',
                                'resultado_pcr_2',
                                'resultado_pcr_3',
                                'fecha_toma_muestra_1',
                                'fecha_toma_muestra_2',
                                'fecha_toma_muestra_3',
                                'fecha_resultado_pcr_1',
                                'fecha_resultado_pcr_2',
                                'fecha_resultado_pcr_3'
    
                               ]]


In [7]:
# Transforma Fechas a formato DateTime 

data_aysen_clean['fecha_notificacion'] = pd.to_datetime(data_aysen_clean['fecha_notificacion']) 

data_aysen_clean['fecha_toma_muestra_1'] = pd.to_datetime(data_aysen_clean['fecha_toma_muestra_1'])

#data_aysen_clean['fecha_toma_muestra_2'] = pd.to_datetime(data_aysen_clean['fecha_toma_muestra_2'])

data_aysen_clean['fecha_toma_muestra_3'] = pd.to_datetime(data_aysen_clean['fecha_toma_muestra_3'])

data_aysen_clean['fecha_resultado_pcr_1'] = pd.to_datetime(data_aysen_clean['fecha_resultado_pcr_1'])

data_aysen_clean['fecha_resultado_pcr_2'] = pd.to_datetime(data_aysen_clean['fecha_resultado_pcr_2'])

data_aysen_clean['fecha_resultado_pcr_3'] = pd.to_datetime(data_aysen_clean['fecha_resultado_pcr_3'])


<ipython-input-7-45c512c70126>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_aysen_clean['fecha_notificacion'] = pd.to_datetime(data_aysen_clean['fecha_notificacion'])
<ipython-input-7-45c512c70126>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_aysen_clean['fecha_toma_muestra_1'] = pd.to_datetime(data_aysen_clean['fecha_toma_muestra_1'])
<ipython-input-7-45c512c70126>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [8]:
# Confirma transformacion a DateTime64 

data_aysen_clean['fecha_notificacion']

0        2020-03-12
1        2020-03-14
2        2020-03-12
3        2020-03-12
4        2020-03-12
            ...    
165356   2021-08-22
165357   2021-08-22
165358   2021-08-22
165359   2021-08-22
165360   2021-08-23
Name: fecha_notificacion, Length: 158806, dtype: datetime64[ns]

## Cálculo de indicadores

In [9]:
semana_x = 33

year = 2021

# Filtra datos para semana_x y año (year)

data_semx = data_aysen_clean[(data_aysen_clean['fecha_notificacion'].dt.year == year)&(data_aysen_clean['semana_epidemiologica']== semana_x)]

data_semx.sample(10)

,numero_folio,comuna_residencia,fecha_notificacion,semana_epidemiologica,etapa_clinica,resultado_pcr_1,resultado_pcr_2,resultado_pcr_3,fecha_toma_muestra_1,fecha_toma_muestra_2,fecha_toma_muestra_3,fecha_resultado_pcr_1,fecha_resultado_pcr_2,fecha_resultado_pcr_3
162279,16065982,Coyhaique,2021-08-16,33,DESCARTADA,NEGATIVO,NEGATIVO,NaN,2021-08-16,2021-07-06,NaT,2021-08-16,2021-07-07,NaT
164305,16191927,Coyhaique,2021-08-18,33,DESCARTADA,NEGATIVO,NaN,NaN,2021-08-18,NaN,NaT,2021-08-20,NaT,NaT
162487,16073639,Coyhaique,2021-08-16,33,BUSQUEDA A,NaN,NaN,NaN,NaT,NaN,NaT,NaT,NaT,NaT
164174,16186387,Coyhaique,2021-08-19,33,DESCARTADA,NEGATIVO,NaN,NaN,2021-08-19,NaN,NaT,2021-08-20,NaT,NaT
164135,16185702,Coyhaique,2021-08-19,33,DESCARTADA,NEGATIVO,NEGATIVO,NaN,2021-08-19,2021-08-01,NaT,2021-08-20,2021-08-01,NaT
161732,16029089,Coyhaique,2021-08-15,33,DESCARTADA,NEGATIVO,NEGATIVO,NaN,2021-04-23,2021-08-15,NaT,2021-04-25,2021-08-16,NaT
163031,16111625,Coyhaique,2021-08-17,33,DESCARTADA,NaN,NaN,NaN,NaT,NaN,NaT,NaT,NaT,NaT
162412,16069993,Coyhaique,2021-08-16,33,DESCARTADA,NaN,NaN,NaN,2021-08-16,NaN,NaT,NaT,NaT,NaT
162301,16066881,Coyhaique,2021-08-16,33,DESCARTADA,NaN,NEGATIVO,NaN,NaT,2021-07-29,NaT,NaT,2021-07-31,NaT
165172,16245003,Aisén,2021-08-21,33,SOSPECHA,NaN,NEGATIVO,NEGATIVO,2021-08-21,2021-08-05,2021-08-21,NaT,2021-08-06,2021-08-22


### Indicador Positividad regional

In [15]:
pcr_pos = data_semx[(data_semx['resultado_pcr_1']=='POSITIVO')|(data_semx['resultado_pcr_2']=='POSITIVO')|(data_semx['resultado_pcr_3']=='POSITIVO')].shape[0]


pcr_tot = data_semx[(data_semx['resultado_pcr_1']=='POSITIVO')|(data_semx['resultado_pcr_2']=='POSITIVO')|(data_semx['resultado_pcr_3']=='POSITIVO')|(data_semx['resultado_pcr_1']=='NEGATIVO')|(data_semx['resultado_pcr_2']=='NEGATIVO')|(data_semx['resultado_pcr_3']=='NEGATIVO')].shape[0]

In [16]:
print('Indicador de positividad regional: ' + str(pcr_pos/pcr_tot*100))

Indicador de positividad regional: 1.4735165272799682


In [17]:
# Muestra datos de pcr realizados en semana_x

data_semx.groupby(['fecha_notificacion', 'resultado_pcr_1'])[['numero_folio']].count().rename(columns={'numero_folio': 'numero_tests'})

numero_tests
fecha_notificacion resultado_pcr_1              
2021-08-15         NEGATIVO                   62
                   POSITIVO                    6
2021-08-16         NEGATIVO                  533
                   NO CONCLUYENTE              2
                   POSITIVO                    3
2021-08-17         NEGATIVO                  348
                   POSITIVO                    9
2021-08-18         NEGATIVO                  358
                   POSITIVO                    2
2021-08-19         NEGATIVO                  398
                   POSITIVO                    1
2021-08-20         NEGATIVO                  349
                   POSITIVO                    1
2021-08-21         NEGATIVO                   55
                   POSITIVO                    1

In [20]:
# Crea gráfico de Positividad regional semana_x

px.bar(data_semx.groupby(['fecha_notificacion', 'resultado_pcr_1'])[['numero_folio']].count().rename(columns={'numero_folio': 'numero_tests'}).reset_index(),
        x= 'fecha_notificacion',
        y= 'numero_tests',
        color= 'resultado_pcr_1'
        
       )

In [21]:
# Agrega 52 semanas a las semanas de 2021 y crea semana_epidemiologica_21

data_aysen_clean1 = data_aysen_clean[data_aysen_clean['fecha_notificacion']<=pd.to_datetime('2021-01-02')]
data_aysen_clean2 = data_aysen_clean[data_aysen_clean['fecha_notificacion']>=pd.to_datetime('2021-01-03')]


data_aysen_clean1['semana_epidemiologica_21'] = data_aysen_clean1['semana_epidemiologica']
data_aysen_clean2['semana_epidemiologica_21'] = data_aysen_clean2['semana_epidemiologica']+53

data_aysen_clean_21 = data_aysen_clean1.append(data_aysen_clean2, ignore_index=True)


<ipython-input-21-cd20b4640465>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-21-cd20b4640465>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [24]:
# Crea gráfico de Resultados PCR regional por semana epidemiologica

px.bar(data_aysen_clean_21.groupby(['semana_epidemiologica_21', 'resultado_pcr_1'])[['numero_folio']].count().rename(columns={'numero_folio': 'numero_tests'}).reset_index(),
        x= 'semana_epidemiologica_21',
        y= 'numero_tests',
        color= 'resultado_pcr_1'
        
       )

In [35]:
data_semepi_pcr = data_aysen_clean_21.groupby(['semana_epidemiologica_21', 'resultado_pcr_1'])[['numero_folio']].count().rename(columns={'numero_folio': 'numero_tests'})

In [66]:
# Test PCR realizados por semana epidemiologica

data_semepi_pcr = data_semepi_pcr.groupby('semana_epidemiologica_21').sum()

data_semepi_pcr

,numero_tests
semana_epidemiologica_21,
10,1
11,6
12,11
13,10
14,13
...,...
83,2254
84,1988
85,2101


In [84]:
px.line(data_semepi_pcr.reset_index(),
        x= 'semana_epidemiologica_21',
        y= 'numero_tests'
)

In [64]:
# Tests positivo por semana epidemiologica

data_positivo = data_semepi_pcr.reset_index().set_index('semana_epidemiologica_21')

data_positivo = data_positivo[data_positivo['resultado_pcr_1']=='POSITIVO'].rename(columns={'numero_tests':'pcr_positivo'}).drop(columns=['resultado_pcr_1'])

In [67]:
data_positivo

,pcr_positivo
semana_epidemiologica_21,
12,1
14,2
19,1
20,1
21,2
...,...
82,16
83,13
84,16


In [83]:
px.line(data_positivo.reset_index(),
        x= 'semana_epidemiologica_21',
        y= 'pcr_positivo'
)

In [69]:
pcr_positividad = data_positivo.merge(data_semepi_pcr, on='semana_epidemiologica_21')

In [73]:
pcr_positividad['positividad'] = pcr_positividad['pcr_positivo']/pcr_positividad['numero_tests']*100

In [74]:
pcr_positividad

,pcr_positivo,numero_tests,positividad
semana_epidemiologica_21,,,
12,1,11,9.090909
14,2,13,15.384615
19,1,7,14.285714
20,1,6,16.666667
21,2,20,10.000000
...,...,...,...
82,16,2284,0.700525
83,13,2254,0.576752
84,16,1988,0.804829


In [86]:
# Crea gráfico de Positividad regional por semana epidemiologica

px.line(pcr_positividad.reset_index(),
        x= 'semana_epidemiologica_21',
        y= 'positividad',
        #color= 'resultado_pcr_1'
        
       )